In [8]:
from tempfile import gettempdir
import biotite
import biotite.structure.io.pdb as pdb
import biotite.database.rcsb as rcsb
import biotite.structure as struc
import numpy as np
import glob
import os
import mdtraj as md
import matplotlib.pyplot as plt

In [ ]:
carpeta_structures = "..\\Experimental\\RNA_predicted\\HBonds\\Proves"
carpeta_resultados = "..\\Experimental\\RNA_predicted\\HBonds\\Proves"

In [4]:

archivos = os.path.join(carpeta_structures, "*.pdb")

with open(os.path.join(carpeta_resultados, "proves.txt"), "w") as f_out:
    for archivo in glob.glob(archivos):
        pdb_file = pdb.PDBFile.read(archivo)
        atom_array = pdb.get_structure(pdb_file)[0]
        nucleotides = atom_array[struc.filter_nucleotides(atom_array)]
        
        # Get the residue names and residue ids of the nucleotides
        residue_ids = []
        residue_names = []
        for residue in struc.residue_iter(nucleotides):
            mapped_nucleotide, exact_match = struc.map_nucleotide(residue)
            if mapped_nucleotide is None:
                continue
            residue_ids.append(residue[0].res_id)
            if exact_match:
                residue_names.append(mapped_nucleotide)
            else:
                residue_names.append(mapped_nucleotide.lower())
        
        
        # Compute the basepairs
        base_pairs = struc.base_pairs(nucleotides)
        glycosidic_bonds = struc.base_pairs_glycosidic_bond(nucleotides, base_pairs)
        edges = struc.base_pairs_edge(nucleotides, base_pairs)
        base_pairs = struc.get_residue_positions(
            nucleotides, base_pairs.flatten()
        ).reshape(base_pairs.shape)

        annotations = []
        for bases, edge_types, orientation in zip(base_pairs, edges, glycosidic_bonds):
            for base, edge in zip(bases, edge_types):
                if orientation == 1:
                    annotation = "c"
                else:
                    annotation = "t"
                if edge == 1:
                    annotation += "W"
                elif edge == 2:
                    annotation += "H"
                else:
                    annotation += "S"
                annotations.append(annotation)

        # File with the base pairings and the Leontis-Westhof classification
        f_out.write(
            f"Archivo: {os.path.basename(archivo)}\n")
                      
        for i in range(base_pairs.shape[0]):
            edge1 = annotations[i*2]
            edge2 = annotations[i*2+1]
            f_out.write(
                f"{base_pairs[i, 0]} {residue_names[base_pairs[i, 0]]}"
                f" {glycosidic_bonds[i]} {edge1} - "
                f"{base_pairs[i, 1]} {residue_names[base_pairs[i, 1]]}"
                f" {glycosidic_bonds[i]} {edge2}\n"
            )


In [7]:
# For trRosetta and 3dRNA i have no problems for H-bond predictions but for DRFold I will use another module
# DRFold seems to be less precise, probably because it does not minimize the energy of the structure

carpeta_structures = "..\\Experimental\\RNA_predicted\\HBonds\\Proves"

archivos = os.path.join(carpeta_structures, "1ajf.pdb")

traj = md.load(archivos)
hbonds = md.baker_hubbard(traj)
print("H Bonds:")
print(hbonds)


traj = md.load(archivos)
hbonds = md.wernet_nilsson(traj)
print("H Bonds:")
print(hbonds)

H Bonds:
[[ 15  28 564]
 [ 17  29 563]
 [ 49  62 536]
 [ 82  92 502]
 [113 126 472]
 [147 160 439]
 [181 194 409]
 [215 228 379]
 [217 229 378]
 [251 263 333]
 [251 264 345]
 [381 391 214]
 [410 422 180]
 [440 452 146]
 [470 482 114]
 [503 516  80]
 [505 517  79]
 [534 546  50]
 [566 577  14]
 [584 594 143]
 [585 597 177]
 [585 598 180]
 [585 598 214]
 [585 599 248]
 [587 603 177]
 [587 605 143]
 [139 157 169]
 [275 292 301]
 [405 420 431]
 [495 513 522]]
H Bonds:
[array([[ 15,  28, 564],
       [ 17,  29, 563],
       [ 49,  62, 536],
       [ 82,  92, 502],
       [113, 126, 472],
       [147, 160, 439],
       [181, 194, 409],
       [215, 228, 379],
       [217, 229, 378],
       [251, 263, 333],
       [251, 264, 345],
       [381, 391, 214],
       [410, 422, 180],
       [440, 452, 146],
       [470, 482, 114],
       [503, 516,  80],
       [505, 517,  79],
       [534, 546,  50],
       [566, 577,  14],
       [585, 597, 177],
       [585, 598, 180],
       [587, 603, 177],
  

c:\Users\guill\anaconda3\envs\master_modcom\Lib\site-packages\mdtraj\formats\pdb\pdbfile.py:200: UserWarning: Unlikely unit cell vectors detected in PDB file likely resulting from a dummy CRYST1 record. Discarding unit cell vectors.
  warnings.warn('Unlikely unit cell vectors detected in PDB file likely '
